In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# hyperparameters
block_size=8
batch_size=4
max_iters = 1000
# eval_interval
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cpu


In [2]:
# open txt file and read the content into text var
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
# character level tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
# train and validation splits
n =int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
# print(x.shape)
print('targets:')
print(y)

inputs:
tensor([[73,  1, 54, 72,  1, 66, 74, 56],
        [66, 69, 54, 67, 78,  1, 66, 58],
        [36, 58, 73,  1, 74, 72,  1, 72],
        [67, 72,  9,  1, 68, 59,  1, 56]])
targets:
tensor([[ 1, 54, 72,  1, 66, 74, 56, 61],
        [69, 54, 67, 78,  1, 66, 58,  1],
        [58, 73,  1, 74, 72,  1, 72, 58],
        [72,  9,  1, 68, 59,  1, 56, 68]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
# inputs and target implementation
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f'when input is {context} target is {target}')

In [7]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:]  # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample form the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled inex to the running sequence 
            index = torch.cat((index, index_next), dim=1)  # (B,T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


Sp_qV'iuJ2BOWW
tc*9;E2P0,3"hO5MVZ1C.0"OJs p"9tQqZsq:*C1C.R7k.TW&PM&ZBymyn
kM"!n?*(U5e[﻿EF'c-:MtrFwHzP,3TpB]fY.nZ2BFcl)qfHFRCB.9qJ
8(]9qf9*fl&zwrkPLmgIR_ltOKOU0k7qMJt-[!9Ubb!:oJZ2XbzDv:qf_d[?3tKeAVD7b7Z7&zP.M:W':VAJ?aQFa_Wuc1'*C17mO8T3?Q. ]X8K!"hA-'1﻿ZxJtKMgUWR&z;m]TOHFRChKKU60VNJW
SznA":*8hOTFB8M Yiw
FRxnd8hBR?a?a(dyC
.zB]BvR?:gZS9SEtaao,xhIs"VYRxGAJ[--.D1uCNl8.mC1.foyCFGlOWWM]JkSPhIL0-vj_Jt_OThTPjdV :!,CKxbUf2al:]boGdGg0ibkPC3;Hf9Wq!rpykUq&_2]ngv_Jj_f9qatjNSzHFpK?61.:V2([F!Rtr4B;gMpe(qhhu:g!"_U


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.746, val loss: 4.751
step: 250, train loss: 4.696, val loss: 4.696
step: 500, train loss: 4.627, val loss: 4.638
step: 750, train loss: 4.567, val loss: 4.598
4.589681625366211


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


M;6"v";XYf0hHk6P"_p4Hxp&?GnBa w5Ts'1KytPY'ebasFM
w['FJ?wZ2sC0-e4lQ [RD5HzUirY-N3r_
u
Au
]75DCzYhmk&z FgX*?6E6mgLHC*C*aAD]WG7J(]aan3Mx9qZ4INYzk.IBboJ?7_l
pd;4p.?T7﻿_nNVjr2aNQvW9k)xU6 9hJ:N"56GK1qNVWG41_M"v3pB8A3(Q"AXXqh5wQ"B&[]P-d JR7﻿teh!n0]p-(&ZSKM2aZriszhc(Q,IYN-(5)cvXzQ'1wBs':!Y0yCzZ&g)
p9hmH!Bjw[oy,0xhUWupm f6Is& mQwZsOYigeBMu()"U028"&QT'irn4yk-!N)AuZajN6k(3?aN]yIR﻿q,,dT---MFHxG&?NWuCNtug3n6&[XDy;S'L_XN]?*1CIOslyCN-Gj[TAcPHkekS_7ZD G2rtAB(bc"yK﻿qi[tn﻿Hg.y:!T!vWG"_XY)g9h,﻿Uq[﻿EEvk,MJ,vK mg:P,
